In [14]:

import os
import json
import time
import base64
import requests

import pandas as pd
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
from datetime import datetime
import time

def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['previous_close'])
    data['low-pc'] = abs(data['low'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean()

    return atr

def supertrend(df, period=10, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1
        
        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df


def get_data(symbol, bar):
    
    d = requests.get(f'https://www.okx.com/api/v5/market/index-candles?instId={symbol}&bar={bar}').json()
    data = pd.DataFrame(d['data'],columns=['timestamp','open','high','low','close','confirm'])
    data['timestamp'] = pd.to_datetime(data['timestamp'],utc=True, unit='ms')
    for i in ['open','high','low','close']:
        data[i] = data[i].astype(float)
    data = data.set_index('timestamp').sort_index()
    return data

def get_current_trend(symbol, bar):
    data = get_data(symbol, bar)
    data = supertrend(data)
    
    return bool(data.iloc[len(data)-1]['in_uptrend'])

In [2]:

from aevo import AevoClient
from clientConfig import CLIENT_CONFIG

from datetime import datetime
from loguru import logger as log



# Add Credentials
aevo = AevoClient(**CLIENT_CONFIG)


In [39]:

def get_price_for_quantity(symbol,direction, quantity):
    orderbook = aevo.get_orderbook(symbol)[direction]
    
    for p,a in orderbook:
        quantity -= float(a)
        if quantity <= 0:
            return p
    return orderbook[20][0]

get_price_for_quantity('ETH-PERP','asks', 5)

'2982.45'

In [40]:
get_price_for_quantity('ETH-PERP','asks', 0)

'2984.19'

In [46]:


def get_price_for_quantity(symbol,direction, quantity):
    orderbook = aevo.get_orderbook(symbol)[direction]
    
    for p,a in orderbook:
        quantity -= float(a)
        if quantity <= 0:
            return p
    return orderbook[20][0]

def clear_existing_positions(symbol_id):
    
    aevo.rest_cancel_all_orders()
    print(f"[CLEAR] clear existing positions {datetime.now().isoformat()}")
    while len(aevo.rest_get_data("positions")['positions'])>0:
        pos = aevo.rest_get_data("positions")['positions'][0]
        side = pos['side']
        pos_amount = float(pos['amount'])

        if side == 'buy':
            aevo.rest_create_market_order(symbol_id,False,pos_amount)
        else:
            aevo.rest_create_market_order(symbol_id,True,pos_amount)
        time.sleep(5)
        aevo.rest_cancel_all_orders()

c = 0
def run_bot(symbol,symbol_id,is_trading_up,amount,amount_for_price_discrepancy):
    
    
    global c
    c+=1
    clear_existing_positions(symbol_id)

    
    print(f"[OREDER] epoch {c} time {datetime.now().isoformat()}")

    direction = 'asks' if is_trading_up else 'bids'
    order = aevo.rest_create_market_order(symbol_id,is_trading_up,amount)
    if 'error' in order:
        raise Exception(order)
    order = aevo.rest_create_order(symbol_id,not is_trading_up, float(get_price_for_quantity(symbol,direction,amount_for_price_discrepancy)),amount) 
    if 'error' in order:
        raise Exception(order)
    
    open_time = datetime.now()

    print(f"opentime {open_time.isoformat()} checking all position filled")

    while len(aevo.rest_get_data("positions")['positions'])>0 or len(aevo.rest_get_open_orders()) > 0 :
        print(f"{datetime.now().isoformat()} position not filled")
        if (datetime.now() - open_time).seconds > 60*20:
            print(f"[TIMEOUT] timeout {datetime.now().isoformat()}")
            
            clear_existing_positions(symbol_id)
            break
        time.sleep(10)

    

while True:
    try:
        run_bot('ETH-PERP',1,get_current_trend('ETH-USDT', '15m'),0.03,30)
    except Exception as e:
        print(f"ERROR {e}")
        clear_existing_positions(1)
        break
    time.sleep(1)

[CLEAR] clear existing positions 2024-02-23T00:19:37.118125


2024-02-23 00:19:37.738 | INFO     | aevo:sign_order:517 - {'name': 'Aevo Mainnet', 'version': '1', 'chainId': '1'}
2024-02-23 00:19:43.759 | INFO     | aevo:sign_order:517 - {'name': 'Aevo Mainnet', 'version': '1', 'chainId': '1'}


[OREDER] epoch 1 time 2024-02-23T00:19:43.759171


2024-02-23 00:19:44.458 | INFO     | aevo:sign_order:517 - {'name': 'Aevo Mainnet', 'version': '1', 'chainId': '1'}
2024-02-23 00:19:44.467 | INFO     | aevo:rest_create_order:196 - {'maker': '0x00000038383f81B2C6153C37053Bd6c3201f42b4', 'is_buy': True, 'instrument': 1, 'limit_price': '2977470000', 'amount': '30000', 'salt': '9326159238', 'signature': '0x2fb55b330af2b1cfa1accb8b2ff50d570ca0e9c2e95b52481b54b162d22a8d0c29bd04e2c183b483ba26fd72ddb9cc18b2954836e349f28c059ab8425822b9f91b', 'post_only': True, 'reduce_only': False, 'close_position': False, 'timestamp': 1708618784}


opentime 2024-02-23T00:19:44.864960 checking all position filled
2024-02-23T00:19:45.182086 position not filled
[CLEAR] clear existing positions 2024-02-23T00:19:57.413318


...

opentime 2024-02-23T00:23:06.643016 checking all position filled
2024-02-23T00:23:06.956424 position not filled
2024-02-23T00:23:17.304538 position not filled
2024-02-23T00:23:27.630228 position not filled
2024-02-23T00:23:37.957407 position not filled
2024-02-23T00:23:48.274876 position not filled
2024-02-23T00:23:58.593555 position not filled
2024-02-23T00:24:08.925871 position not filled
2024-02-23T00:24:19.262360 position not filled
2024-02-23T00:24:29.593339 position not filled
2024-02-23T00:24:39.921781 position not filled
2024-02-23T00:24:50.238143 position not filled
2024-02-23T00:25:00.567730 position not filled
2024-02-23T00:25:10.903062 position not filled
2024-02-23T00:25:21.246118 position not filled
2024-02-23T00:25:31.558690 position not filled
2024-02-23T00:25:41.878424 position not filled
2024-02-23T00:25:52.196675 position not filled
2024-02-23T00:26:02.527920 position not filled
2024-02-23T00:26:12.871225 position not filled
2024-02-23T00:26:23.214863 position not fi

2024-02-23 00:28:08.707 | INFO     | aevo:sign_order:517 - {'name': 'Aevo Mainnet', 'version': '1', 'chainId': '1'}


[OREDER] epoch 12 time 2024-02-23T00:28:08.707747


2024-02-23 00:28:09.425 | INFO     | aevo:sign_order:517 - {'name': 'Aevo Mainnet', 'version': '1', 'chainId': '1'}
2024-02-23 00:28:09.435 | INFO     | aevo:rest_create_order:196 - {'maker': '0x00000038383f81B2C6153C37053Bd6c3201f42b4', 'is_buy': True, 'instrument': 1, 'limit_price': '2962880000', 'amount': '30000', 'salt': '9501867489', 'signature': '0x1b8ced08c475864dbc98aa6263543860033881aa9fc594f0ba663e4934c7b4317afa372f9f7dea2c80ec2e4015cb651edbc308bd7a6ab9283a06d188be52bf9f1c', 'post_only': True, 'reduce_only': False, 'close_position': False, 'timestamp': 1708619289}


opentime 2024-02-23T00:28:09.750988 checking all position filled
2024-02-23T00:28:10.061938 position not filled
2024-02-23T00:28:20.410621 position not filled
2024-02-23T00:28:30.719185 position not filled
2024-02-23T00:28:41.036122 position not filled
2024-02-23T00:28:51.355636 position not filled
2024-02-23T00:29:01.759006 position not filled
2024-02-23T00:29:12.065893 position not filled
2024-02-23T00:29:22.375998 position not filled
2024-02-23T00:29:32.690332 position not filled
2024-02-23T00:29:43.027964 position not filled
2024-02-23T00:29:53.345615 position not filled
2024-02-23T00:30:03.701638 position not filled
2024-02-23T00:30:14.018793 position not filled
2024-02-23T00:30:24.337381 position not filled
2024-02-23T00:30:34.656815 position not filled
2024-02-23T00:30:44.981128 position not filled
2024-02-23T00:30:55.297872 position not filled
2024-02-23T00:31:05.641752 position not filled
2024-02-23T00:31:15.956641 position not filled
2024-02-23T00:31:26.269723 position not fi

In [ ]:
# import schedule
# def clear_all_positions(symbol_id):
#     print(f"[CLEAR] clear existing positions {datetime.now().isoformat()}")
#     aevo.rest_cancel_all_orders()
#     while len(aevo.rest_get_data("positions")['positions'])>0:
#         pos = aevo.rest_get_data("positions")['positions'][0]
#         side = pos['side']
#         pos_amount = float(pos['amount'])

#         if side == 'buy':
#             aevo.rest_create_market_order(symbol_id,False,pos_amount)
#         else:
#             aevo.rest_create_market_order(symbol_id,True,pos_amount)
#         time.sleep(5)
#         aevo.rest_cancel_all_orders()

# def check_buy_sell_signals(df):
#     global in_position

#     print("checking for buy and sell signals")
#     # print(df.tail(5))
#     last_row_index = len(df.index) - 1
#     previous_row_index = last_row_index - 1

#     if not df['in_uptrend'][previous_row_index] and df['in_uptrend'][last_row_index]:
#         print("changed to uptrend, buy")
#         if not in_position:
#             clear_all_positions(5197)
#             print("[BUY ORDER]")
#             aevo.rest_create_market_order(5197,True,10)
            
#             in_position = True
#         else:
#             print("already in position, nothing to do")
    
#     if df['in_uptrend'][previous_row_index] and not df['in_uptrend'][last_row_index]:
#         if in_position:
#             print("changed to downtrend, sell")
#             clear_all_positions(5197)
#             print("[SELL ORDER]\n")
#             aevo.rest_create_market_order(5197,False,10)
#             in_position = False
#         else:
#             print("You aren't in position, nothing to sell")

# def run_bot():
#     print(f"Fetching new bars for {datetime.now().isoformat()}")
#     data = get_data('SOL-USDT', '4h')
#     supertrend_data = supertrend(data)
#     check_buy_sell_signals(supertrend_data)


# schedule.every(10).seconds.do(run_bot)


# while True:
#     schedule.run_pending()
#     time.sleep(1)
